<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [82]:

from Bert.pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from Bert.pytorch_pretrained_bert.modeling import BertForQuestionAnswering, BertConfig, WEIGHTS_NAME, CONFIG_NAME
from Bert.pytorch_pretrained_bert.optimization import BertAdam, warmup_linear
from Bert.pytorch_pretrained_bert.tokenization import (BasicTokenizer,
                                                  BertTokenizer,
                                                  whitespace_tokenize)

In [88]:

import torch
tokenizer = torch.hub.load('huggingface/pytorch-pretrained-BERT', 'bertTokenizer', 'bert-base-cased', do_basic_tokenize=False)

# Tokenized input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)

Downloading: "https://github.com/huggingface/pytorch-pretrained-BERT/archive/master.zip" to /home/mouadh/.cache/torch/hub/master.zip
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
100%|██████████| 213450/213450 [00:01<00:00, 121878.09B/s]


In [89]:
tokenized_text

['[CLS]',
 'Who',
 'was',
 'Jim',
 'He',
 '##nson',
 '?',
 '[SEP]',
 'Jim',
 'He',
 '##nson',
 'was',
 'a',
 'puppet',
 '##eer',
 '[SEP]']

In [90]:
tokenizer = BertTokenizer.from_pretrained(
    "bert-base-multilingual-cased", do_lower_case=False)
tokenizer.tokenize("helo")

['hel', '##o']

In [66]:
! pip3 install tokenize


Exception:
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/pip/basecommand.py", line 215, in main
    status = self.run(options, args)
  File "/usr/lib/python3/dist-packages/pip/commands/install.py", line 353, in run
    wb.build(autobuilding=True)
  File "/usr/lib/python3/dist-packages/pip/wheel.py", line 749, in build
    self.requirement_set.prepare_files(self.finder)
  File "/usr/lib/python3/dist-packages/pip/req/req_set.py", line 380, in prepare_files
    ignore_dependencies=self.ignore_dependencies))
  File "/usr/lib/python3/dist-packages/pip/req/req_set.py", line 554, in _prepare_file
    require_hashes
  File "/usr/lib/python3/dist-packages/pip/req/req_install.py", line 278, in populate_link
    self.link = finder.find_requirement(self, upgrade)
  File "/usr/lib/python3/dist-packages/pip/index.py", line 465, in find_requirement
    all_candidates = self.find_all_candidates(req.name)
  File "/usr/lib/python3/dist-packages/pip/index.py", line 423, in fin

In [9]:
x = torch.randn(4, 4)
x.size()

torch.Size([4, 4])

In [12]:
x

tensor([[ 0.0092,  1.2476, -0.6837,  2.0582],
        [ 1.2154, -0.0888,  0.6328,  1.1187],
        [ 0.6453,  0.0379,  1.2124,  0.6464],
        [-0.7502,  0.5771,  0.7226,  0.4296]])

In [38]:
x.view(-1, 16)

tensor([[ 0.0092,  1.2476, -0.6837,  2.0582,  1.2154, -0.0888,  0.6328,  1.1187,
          0.6453,  0.0379,  1.2124,  0.6464, -0.7502,  0.5771,  0.7226,  0.4296]])

In [47]:
x = torch.zeros(2, 1, 2, 1, 2)
x.size()

torch.Size([2, 1, 2, 1, 2])

In [54]:
torch.squeeze(x).size()

torch.Size([2, 2, 2])

In [48]:
import json
import sys
import os
import nltk
import pandas as pd
import numpy as np
import json
import sys
import os
import nltk

In [67]:
train = pd.read_csv('train.csv', index_col=0)

In [68]:
train.head()

,title,context,ids,question,is_impossible,answer_start,text,answer_end
0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,False,269,in the late 1990s,286
1,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,False,207,singing and dancing,226
2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,False,526,2003,530
3,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,False,166,"Houston, Texas",180
4,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,False,276,late 1990s,286


We will transform context to a sequence of sentences

In [69]:
sentences = nltk.sent_tokenize(train.loc[0]['context'])

In [70]:
sentences

['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress.',
 "Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child.",
 "Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time.",
 'Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".']

In [71]:
# New column contains the tokenized sentences
train['context_seq'] = train['context'].map(lambda c: nltk.sent_tokenize(c))

In [72]:
train.head()

,title,context,ids,question,is_impossible,answer_start,text,answer_end,context_seq
0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,False,269,in the late 1990s,286,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...
1,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,False,207,singing and dancing,226,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...
2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,False,526,2003,530,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...
3,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,False,166,"Houston, Texas",180,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...
4,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,False,276,late 1990s,286,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...


In [73]:
train.dropna(inplace=True)

In [74]:
tokenized_train = train[['context_seq', 'question', 'text']]

In [75]:
tokenized_train.to_csv('tokenized_train.csv')

In [76]:
# We droped this column this it's only one 
train[train['text'].isna()]

,title,context,ids,question,is_impossible,answer_start,text,answer_end,context_seq
